In [ ]:
!apt-get install poppler-utils
!pip install PyMuPDF

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 0s (907 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 56.2 MB/s eta 0:00:00


In [ ]:
!pip install pdf2image pillow

In [ ]:
import os
import json
import csv
import pandas as pd
from pathlib import Path
import base64
from io import BytesIO
from PIL import Image
import pdf2image
from openai import OpenAI
from google.colab import userdata
from typing import Dict, List, Optional
import time
import logging
from datetime import datetime

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ClientDataExtractor:
    def __init__(self, api_key: str):
        """
        Initialize the Client Data Extractor with GPT-4 Vision

        Args:
            api_key: OpenAI API key
        """
        self.client = OpenAI(api_key=api_key)
        self.metrics = [
            "page_number", "date", "Timestamp", "Client_Id",
            "Company_Name", "Address", "Contact/Attorneys",
            "Phone", "Fax1", "Fax2", "Send Fx1", "Send Fx2", "Status", "Email",
            "Default_Warrant", "Manager", "Billing_Type", "A/R_Balance"
        ]

    def pdf_to_images(self, pdf_path: str, max_pages: int = 20) -> List[Image.Image]:
        """
        Convert PDF pages to images with multiple fallback methods

        Args:
            pdf_path: Path to the PDF file
            max_pages: Maximum number of pages to process (to control costs)

        Returns:
            List of PIL Images
        """
        print(f"\n=== Converting PDF to Images: {pdf_path} ===")
        print(f"File exists: {os.path.exists(pdf_path)}")
        print(f"File size: {os.path.getsize(pdf_path) / (1024*1024):.2f} MB")

        # Method 1: Standard pdf2image
        try:
            print("Attempting Method 1: Standard pdf2image...")
            images = pdf2image.convert_from_path(
                pdf_path,
                dpi=300,
                first_page=1,
                last_page=max_pages,
                fmt='PNG'
            )

            if images:
                print(f"✅ Method 1 Success: Converted {len(images)} pages")
                return images
            else:
                print("❌ Method 1: No images returned")

        except Exception as e:
            print(f"❌ Method 1 Error: {str(e)}")

        # Method 2: Try with different parameters
        try:
            print("Attempting Method 2: Lower DPI and different settings...")
            images = pdf2image.convert_from_path(
                pdf_path,
                dpi=200,  # Lower DPI
                first_page=1,
                last_page=min(5, max_pages),  # Fewer pages initially
                fmt='JPEG',
                thread_count=1,
                use_pdftocairo=False
            )

            if images:
                print(f"✅ Method 2 Success: Converted {len(images)} pages")
                return images
            else:
                print("❌ Method 2: No images returned")

        except Exception as e:
            print(f"❌ Method 2 Error: {str(e)}")

        # Method 3: Try with pdftocairo
        try:
            print("Attempting Method 3: Using pdftocairo...")
            images = pdf2image.convert_from_path(
                pdf_path,
                dpi=150,
                first_page=1,
                last_page=3,  # Just first 3 pages
                use_pdftocairo=True
            )

            if images:
                print(f"✅ Method 3 Success: Converted {len(images)} pages")
                return images
            else:
                print("❌ Method 3: No images returned")

        except Exception as e:
            print(f"❌ Method 3 Error: {str(e)}")

        # Method 4: Try with PyMuPDF as fallback
        try:
            print("Attempting Method 4: PyMuPDF fallback...")
            import fitz  # PyMuPDF

            doc = fitz.open(pdf_path)
            images = []

            for page_num in range(min(max_pages, len(doc))):
                page = doc.load_page(page_num)
                mat = fitz.Matrix(2, 2)  # 2x zoom
                pix = page.get_pixmap(matrix=mat)
                img_data = pix.tobytes("png")
                img = Image.open(BytesIO(img_data))
                images.append(img)

                if len(images) >= 5:  # Limit for testing
                    break

            doc.close()

            if images:
                print(f"✅ Method 4 Success: Converted {len(images)} pages using PyMuPDF")
                return images
            else:
                print("❌ Method 4: No images returned")

        except ImportError:
            print("❌ Method 4: PyMuPDF not installed (pip install PyMuPDF)")
        except Exception as e:
            print(f"❌ Method 4 Error: {str(e)}")

        print("❌ All methods failed to convert PDF to images")
        return []

    def encode_image(self, image: Image.Image) -> str:
        """
        Encode PIL Image to base64 string

        Args:
            image: PIL Image object

        Returns:
            Base64 encoded string
        """
        try:
            # Resize image if too large (to control API costs)
            max_size = (2048, 2048)
            if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
                image.thumbnail(max_size, Image.Resampling.LANCZOS)

            buffer = BytesIO()
            image.save(buffer, format='PNG')
            buffer.seek(0)

            return base64.b64encode(buffer.getvalue()).decode('utf-8')

        except Exception as e:
            logger.error(f"Error encoding image: {str(e)}")
            return ""

    def create_vision_extraction_prompt(self, document_name: str = "") -> str:
        """
        Create a structured prompt for client data extraction using vision

        Args:
            document_name: Name of the document (optional)

        Returns:
            Formatted prompt string
        """
        prompt = f"""
You are a professional data analyst analyzing business documents. Extract ALL client and business information entries from the images provided.

Document: {document_name if document_name else "Identify from document"}

Look for client information, contact details, business data, and account information in these images. EXTRACT ALL ENTRIES - do not focus on just one entry.

Extract the following data fields for EVERY ENTRY found and return them in JSON format:

Required Fields for Each Entry:
- page_number: The page number where information is found
- date: Any date mentioned in the document (format: YYYY-MM-DD if possible)
- Timestamp: Any timestamp or date/time information
- Client_Id: Client ID, Account Number, or any unique identifier
- Company_Name: Company or business name
- Address: Full address (street, city, state, zip/postal code)
- Contact/Attorneys: Contact person name or attorney name
- Phone: Phone number (include area code)
- Fax1: Primary fax number
- Fax2: Secondary fax number (if available)
- Send Fx1: Send fax for primary fax number
- Send Fx2: Send for secondary fax number
- Status: Account status, case status, or any status information
- Email: Email address
- Default_Warrant: Default warrant information or legal notes
- Manager: Manager name or account manager
- Billing_Type: Billing type, payment method, or billing information
- A/R_Balance: Accounts receivable balance or outstanding amount

CRITICAL Instructions:
1. Extract EVERY SINGLE ENTRY found in the document - do not skip any
2. Look for tables with multiple rows - each row is typically a separate entry
3. Look for lists, forms with multiple sections, or repeated patterns
4. If you find a table with 10 rows of data, extract all 10 entries
5. Each entry should be a complete set of the required fields
6. For addresses, combine all address components into one field per entry
7. For phone/fax numbers, include area codes and formatting as shown
8. If a field is not visible for a specific entry, use "Not Found" for that field only
9. Pay attention to labels, headers, and field names that might indicate the data type
10. For monetary amounts (A/R_Balance), include currency symbol if shown
11. Number each entry sequentially (entry_1, entry_2, etc.)

Return the data in this exact JSON format for MULTIPLE ENTRIES:
{{
    "document_info": {{
        "document_name": "{document_name}",
        "extraction_timestamp": "{datetime.now().isoformat()}",
        "total_pages_processed": "number of pages analyzed",
        "total_entries_found": "number of entries extracted"
    }},
    "all_entries": [
        {{
            "entry_id": "entry_1",
            "page_number": "value or Not Found",
            "date": "value or Not Found",
            "Timestamp": "value or Not Found",
            "Client_Id": "value or Not Found",
            "Company_Name": "value or Not Found",
            "Address": "value or Not Found",
            "Contact/Attorneys": "value or Not Found",
            "Phone": "value or Not Found",
            "Fax1": "value or Not Found",
            "Fax2": "value or Not Found",
            "Send Fx1": "value or Not Found",
            "Send Fx2": "value or Not Found",
            "Status": "value or Not Found",
            "Email": "value or Not Found",
            "Default_Warrant": "value or Not Found",
            "Manager": "value or Not Found",
            "Billing_Type": "value or Not Found",
            "A/R_Balance": "value or Not Found"
        }},
        {{
            "entry_id": "entry_2",
            "page_number": "value or Not Found",
            "date": "value or Not Found",
            "Timestamp": "value or Not Found",
            "Client_Id": "value or Not Found",
            "Company_Name": "value or Not Found",
            "Address": "value or Not Found",
            "Contact/Attorneys": "value or Not Found",
            "Phone": "value or Not Found",
            "Fax1": "value or Not Found",
            "Fax2": "value or Not Found",
            "Send Fx1": "value or Not Found",
            "Send Fx2": "value or Not Found",
            "Status": "value or Not Found",
            "Email": "value or Not Found",
            "Default_Warrant": "value or Not Found",
            "Manager": "value or Not Found",
            "Billing_Type": "value or Not Found",
            "A/R_Balance": "value or Not Found"
        }}
    ],
    "confidence_score": "high/medium/low",
    "notes": "Total entries found and any relevant notes about data extraction"
}}

IMPORTANT: If only one entry exists, still use the array format with one object. If no entries are found, return an empty array for "all_entries".

Analyze all the provided images carefully and extract EVERY client data entry. Look for patterns in forms, tables, or structured layouts that contain this information. Do not limit yourself to just the first or most complete entry.
"""
        return prompt

    def extract_client_data_from_images(self, images: List[Image.Image], document_name: str = "") -> Dict:
        """
        Use GPT-4 Vision to extract client data from PDF images

        Args:
            images: List of PIL Images from PDF pages
            document_name: Name of the document

        Returns:
            Dictionary containing extracted client data
        """
        try:
            if not images:
                logger.error("No images provided for extraction")
                return self._create_empty_response(document_name, "No images provided")

            print(f"Processing {len(images)} images for {document_name}")

            # Prepare images for API call
            image_messages = []
            for i, image in enumerate(images):
                print(f"Encoding image {i+1}/{len(images)} - Size: {image.size}")
                encoded_image = self.encode_image(image)
                if encoded_image:
                    image_messages.append({
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{encoded_image}",
                            "detail": "high"  # Use high detail for better text recognition
                        }
                    })
                    print(f"Successfully encoded image {i+1}")
                else:
                    print(f"Failed to encode image {i+1}")

                # Limit number of images to control costs
                if len(image_messages) >= 10:  # Increased limit for client documents
                    logger.info(f"Limited to first 10 pages for cost control")
                    break

            if not image_messages:
                logger.error("No images could be encoded")
                return self._create_empty_response(document_name, "No images could be encoded")

            print(f"Prepared {len(image_messages)} images for API call")

            # Create the prompt
            prompt = self.create_vision_extraction_prompt(document_name)

            # Prepare messages for API call
            messages = [
                {
                    "role": "system",
                    "content": "You are a professional data analyst with expertise in reading business documents, legal documents, client files, and extracting structured information accurately from various document types including forms, tables, and reports."
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        *image_messages
                    ]
                }
            ]

            print("Making API call to GPT-4 Vision...")

            # Make API call to GPT-4 Vision
            response = self.client.chat.completions.create(
                model="gpt-4o",  # Use GPT-4 Vision model
                messages=messages,
                max_tokens=4000,  # Increased token limit for multiple entries
                temperature=0  # Low temperature for consistent extraction
            )

            response_text = response.choices[0].message.content
            print(f"Received response from API (length: {len(response_text)})")
            print(f"Response preview: {response_text[:500]}...")

            # Try to parse JSON response
            try:
                # Find JSON in response (in case there's additional text)
                start_idx = response_text.find('{')
                end_idx = response_text.rfind('}') + 1

                if start_idx == -1 or end_idx == 0:
                    logger.error("No JSON found in response")
                    print(f"Full response: {response_text}")
                    return self._create_empty_response(document_name, "No JSON in API response")

                json_str = response_text[start_idx:end_idx]
                print(f"Extracted JSON string: {json_str[:200]}...")

                client_data = json.loads(json_str)
                logger.info(f"Successfully extracted client data for {document_name}")
                return client_data

            except json.JSONDecodeError as je:
                logger.error(f"Failed to parse JSON response for {document_name}: {str(je)}")
                print(f"JSON Error: {str(je)}")
                print(f"Attempted to parse: {json_str[:500]}...")
                return self._create_empty_response(document_name, f"JSON parse error: {str(je)}")

        except Exception as e:
            logger.error(f"Error in GPT-4 Vision extraction for {document_name}: {str(e)}")
            print(f"Full error details: {str(e)}")
            import traceback
            traceback.print_exc()
            return self._create_empty_response(document_name, f"API error: {str(e)}")

    def _create_empty_response(self, document_name: str, error_reason: str = "Extraction failed") -> Dict:
        """Create empty response structure when extraction fails"""
        return {
            "document_info": {
                "document_name": document_name,
                "extraction_timestamp": datetime.now().isoformat(),
                "total_pages_processed": "0",
                "total_entries_found": "0"
            },
            "all_entries": [],
            "confidence_score": "low",
            "notes": error_reason
        }

    def process_multiple_pdfs(self, pdf_directory: str, output_csv: str = "client_data.csv", max_pages_per_pdf: int = 20) -> List[Dict]:
        """
        Process multiple PDF files and extract client data using GPT-4 Vision

        Args:
            pdf_directory: Directory containing PDF files
            output_csv: Output CSV file name
            max_pages_per_pdf: Maximum pages to process per PDF (cost control)

        Returns:
            List of extracted client data dictionaries
        """
        pdf_files = list(Path(pdf_directory).glob("*.pdf"))
        all_client_data = []

        logger.info(f"Found {len(pdf_files)} PDF files to process")

        for pdf_file in pdf_files:
            logger.info(f"Processing: {pdf_file.name}")

            # Use the actual filename as document name
            document_name = pdf_file.name

            # Convert PDF to images
            images = self.pdf_to_images(str(pdf_file), max_pages_per_pdf)

            if images:
                # Extract client data using GPT-4 Vision
                client_data = self.extract_client_data_from_images(images, document_name)
                all_client_data.append(client_data)

                # Add delay to avoid rate limiting (GPT-4V has stricter limits)
                time.sleep(5)
            else:
                logger.warning(f"No images extracted from {pdf_file.name}")
                all_client_data.append(self._create_empty_response(document_name, f"No images extracted from {pdf_file.name}"))

        # Save to CSV
        self.save_to_csv(all_client_data, output_csv)

        return all_client_data

    def save_to_csv(self, client_data_list: List[Dict], output_csv: str):
        """
        Save extracted client data to CSV file - handles multiple entries per document

        Args:
            client_data_list: List of client data dictionaries
            output_csv: Output CSV filename
        """
        try:
            # Prepare data for CSV
            csv_data = []

            for data in client_data_list:
                # Get document info
                doc_info = data.get('document_info', {})
                all_entries = data.get('all_entries', [])

                # If no entries found, create one row with document info
                if not all_entries:
                    row = {
                        'Document Name': doc_info.get('document_name', ''),
                        'Extraction Timestamp': doc_info.get('extraction_timestamp', ''),
                        'Pages Processed': doc_info.get('total_pages_processed', ''),
                        'Total Entries Found': doc_info.get('total_entries_found', '0'),
                        'Entry ID': 'No entries found',
                        'Confidence Score': data.get('confidence_score', ''),
                        'Notes': data.get('notes', '')
                    }

                    # Add all the required client data fields with "Not Found"
                    for field in self.metrics:
                        row[field] = 'Not Found'

                    csv_data.append(row)
                else:
                    # Create a row for each entry found
                    for entry in all_entries:
                        row = {
                            'Document Name': doc_info.get('document_name', ''),
                            'Extraction Timestamp': doc_info.get('extraction_timestamp', ''),
                            'Pages Processed': doc_info.get('total_pages_processed', ''),
                            'Total Entries Found': doc_info.get('total_entries_found', str(len(all_entries))),
                            'Entry ID': entry.get('entry_id', ''),
                            'Confidence Score': data.get('confidence_score', ''),
                            'Notes': data.get('notes', '')
                        }

                        # Add all the required client data fields for this entry
                        for field in self.metrics:
                            row[field] = entry.get(field, 'Not Found')

                        csv_data.append(row)

            # Create DataFrame and save to CSV
            df = pd.DataFrame(csv_data)
            df.to_csv(output_csv, index=False)

            logger.info(f"Client data saved to {output_csv}")

            # Print summary
            total_entries = sum(len(data.get('all_entries', [])) for data in client_data_list)
            print(f"\nProcessing Summary:")
            print(f"Total documents processed: {len(client_data_list)}")
            print(f"Total entries extracted: {total_entries}")
            print(f"Average entries per document: {total_entries/len(client_data_list):.1f}")
            print(f"Output saved to: {output_csv}")
            print(f"\nColumns in output CSV:")
            for col in df.columns:
                print(f"  - {col}")

            # Print extraction statistics
            successful_extractions = sum(1 for data in client_data_list if data.get('confidence_score') != 'low')
            documents_with_entries = sum(1 for data in client_data_list if len(data.get('all_entries', [])) > 0)

            print(f"\nExtraction Statistics:")
            print(f"  - Documents with successful extractions: {successful_extractions}/{len(client_data_list)}")
            print(f"  - Documents with entries found: {documents_with_entries}/{len(client_data_list)}")
            print(f"  - Success rate: {(successful_extractions/len(client_data_list)*100):.1f}%")

            # Show entry distribution
            entry_counts = [len(data.get('all_entries', [])) for data in client_data_list]
            if entry_counts:
                print(f"  - Min entries per document: {min(entry_counts)}")
                print(f"  - Max entries per document: {max(entry_counts)}")

        except Exception as e:
            logger.error(f"Error saving to CSV: {str(e)}")
            import traceback
            traceback.print_exc()

def main():
    """
    Main function to demonstrate usage
    """
    # Initialize the extractor
    API_KEY = userdata.get('OpenApi')
    extractor = ClientDataExtractor(API_KEY)

    # Process PDFs from a directory
    pdf_directory = "/content/drive/MyDrive/Client_Documents"
    output_file = "extracted_client_data_update.csv"

    # Create sample directory structure
    os.makedirs(pdf_directory, exist_ok=True)

    print(f"Place your client PDF documents in the '{pdf_directory}' directory")
    print("The system will extract the following client information:")
    print("  - page_number, date, Timestamp, Client_Id")
    print("  - Company_Name, Address, Contact/Attorneys")
    print("  - Phone, Fax1, Fax2, Send Fx, Status, Email")
    print("  - Default_Warrant, Manager, Billing_Type, A/R_Balance")
    print("\n⚠️  Note: GPT-4 Vision is more expensive than text models. Processing is limited to 20 pages per PDF by default.")

    # Check if directory has PDFs
    pdf_files = list(Path(pdf_directory).glob("*.pdf"))

    if pdf_files:
        print(f"\nFound {len(pdf_files)} PDF files:")
        for pdf in pdf_files:
            print(f"  - {pdf.name}")

        # Process the PDFs
        extracted_data = extractor.process_multiple_pdfs(
            pdf_directory,
            output_file,
            max_pages_per_pdf=20  # Adjust based on your needs and budget
        )

        print(f"\n✅ Processing complete! Check {output_file} for results.")

    else:
        print(f"\n⚠️  No PDF files found in '{pdf_directory}' directory")
        print("Please add your client PDF documents and run again.")

if __name__ == "__main__":
    main()

Place your client PDF documents in the '/content/drive/MyDrive/Client_Documents' directory
The system will extract the following client information:
  - page_number, date, Timestamp, Client_Id
  - Company_Name, Address, Contact/Attorneys
  - Phone, Fax1, Fax2, Send Fx, Status, Email
  - Default_Warrant, Manager, Billing_Type, A/R_Balance

⚠️  Note: GPT-4 Vision is more expensive than text models. Processing is limited to 20 pages per PDF by default.

Found 1 PDF files:
  - page 1 client files.pdf

=== Converting PDF to Images: /content/drive/MyDrive/Client_Documents/page 1 client files.pdf ===
File exists: True
File size: 0.08 MB
Attempting Method 1: Standard pdf2image...
✅ Method 1 Success: Converted 1 pages
Processing 1 images for page 1 client files.pdf
Encoding image 1/1 - Size: (2550, 3301)
Successfully encoded image 1
Prepared 1 images for API call
Making API call to GPT-4 Vision...
Received response from API (length: 6906)
Response preview: ```json
{
    "document_info": {
     